In [338]:
import pandas as pd

In [339]:
f = 'data/cycle-soup.csv'
df = pd.read_csv(f)

In [340]:
# Make sure all pages were downloaded - should be 9 listings / page
df.page.value_counts().mean()

9.0

In [341]:
df['year']       = df['year_brand'].str[:4].astype(int)
df['brand']      = df['year_brand'].str[5:]
df['listing_id'] = df['id'].str.extract("((?<=exvehicles\/).*?(?=\?udealer_id))")
df['dealer_id']  = df['id'].str.extract("((?<=udealer_id\=).*?(?=\&origin_path))")
df['Mileage']    = df['Mileage'].str.extract('(^\d*)').astype(int)
df['lat']        = df['lat'].round(6)
df['lon']        = (df['lon'] * -1).round(6)
df['geom']       = df['lat'].map(str) + ',' + df['lon'].map(str)

df.drop(columns=['year_brand', 'id'],inplace=True)

In [346]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7484 entries, 0 to 9997
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Mileage     7484 non-null   int64  
 1   Condition   7484 non-null   object 
 2   Type        7484 non-null   object 
 3   Color       6119 non-null   object 
 4   Engine      7484 non-null   int64  
 5   page        7484 non-null   int64  
 6   lat         7484 non-null   float64
 7   lon         7484 non-null   float64
 8   model       7484 non-null   object 
 9   price       7484 non-null   object 
 10  year        7484 non-null   int64  
 11  brand       7484 non-null   object 
 12  listing_id  7484 non-null   object 
 13  dealer_id   7484 non-null   object 
 14  geom        7484 non-null   object 
dtypes: float64(2), int64(4), object(9)
memory usage: 935.5+ KB


In [347]:
# filter out "Call For Price"
mask = df['price'].str.contains('[a-zA-Z]')
df = df[~mask]

# Add location data

In [353]:
locs = pd.read_csv('data/locations.csv')

In [354]:
locs.head()

,index,county,state,postcode
0,"42.885832,-88.218567",Waukesha County,Wisconsin,53103
1,"44.864078,-93.441708",Hennepin County,Minnesota,55344-3602
2,"29.689799,-95.526602",Harris County,Texas,77074
3,"29.900771,-95.389418",Harris County,Texas,77037
4,"30.001752,-95.44742",Harris County,Texas,77090


In [355]:
df.shape

(7484, 15)

In [356]:
# add location data to df
df = df.merge(locs, left_on="geom", right_on="index")

In [357]:
# check there are no nulls in added cols
df['county'].isnull().sum()

0

In [358]:
# lowercase all columns
df.columns = [x.lower() for x in df.columns]

In [359]:
df.head()

,mileage,condition,type,color,engine,page,lat,lon,model,price,year,brand,listing_id,dealer_id,geom,index,county,state,postcode
0,30225,Used,Street,Black,0,1,42.885832,-88.218567,FXS,6899,2006,Harley Davidson,077cda29357646746e1a91197b14c2a4ee85d539,597c22c225ec4e0004de5928,"42.885832,-88.218567","42.885832,-88.218567",Waukesha County,Wisconsin,53103
1,2621,Used,Street,BLACK,0,1,42.885832,-88.218567,XG500,3999,2015,Harley Davidson,d47489df7d39a4308c462ffc897daea7a3c652de,597c22c225ec4e0004de5928,"42.885832,-88.218567","42.885832,-88.218567",Waukesha County,Wisconsin,53103
2,6314,Used,Street,Black,0,1,42.885832,-88.218567,XG750,4799,2015,Harley Davidson,4c5d3fa5a288945b02cedb83923e930117b3e261,597c22c225ec4e0004de5928,"42.885832,-88.218567","42.885832,-88.218567",Waukesha County,Wisconsin,53103
3,22433,Used,Street,Black/Orange,0,1,42.885832,-88.218567,XL1200,4499,2008,Harley Davidson,260c8e7fee9942c1e67c2852692ea79eed7dd977,597c22c225ec4e0004de5928,"42.885832,-88.218567","42.885832,-88.218567",Waukesha County,Wisconsin,53103
4,22856,Used,Street,Black/Flames,0,1,42.885832,-88.218567,XL1200,3999,2008,Harley Davidson,fdeb6e9843561a75c56a233e101484d17b82219b,597c22c225ec4e0004de5928,"42.885832,-88.218567","42.885832,-88.218567",Waukesha County,Wisconsin,53103


In [360]:
df['condition'].value_counts()

New     5108
Used    2376
Name: condition, dtype: int64

In [361]:
cols = ['condition', 'type', 'year', 'brand', 'model', 'state', 'mileage', 'price']
df = df.filter(items=cols)

In [362]:
# add model newness
df.head()

,condition,type,year,brand,model,state,mileage,price
0,Used,Street,2006,Harley Davidson,FXS,Wisconsin,30225,6899
1,Used,Street,2015,Harley Davidson,XG500,Wisconsin,2621,3999
2,Used,Street,2015,Harley Davidson,XG750,Wisconsin,6314,4799
3,Used,Street,2008,Harley Davidson,XL1200,Wisconsin,22433,4499
4,Used,Street,2008,Harley Davidson,XL1200,Wisconsin,22856,3999


In [364]:
df = df.mask(df['brand'].map(df['brand'].value_counts()) < 100, 'other')
df['brand'].value_counts()

Harley Davidson    1404
Honda              1272
Suzuki              929
Yamaha              614
other               487
Kawasaki            446
Ducati              432
Triumph             336
Can Am              298
Kymco               296
KTM                 292
Vespa               243
BMW                 219
Genuine Scooter     216
Name: brand, dtype: int64

In [365]:
df.dtypes

condition    object
type         object
year         object
brand        object
model        object
state        object
mileage      object
price        object
dtype: object

In [368]:
df.head()

,condition,type,year,brand,model,state,mileage,price
0,Used,Street,2006,Harley Davidson,FXS,Wisconsin,30225,6899
1,Used,Street,2015,Harley Davidson,XG500,Wisconsin,2621,3999
2,Used,Street,2015,Harley Davidson,XG750,Wisconsin,6314,4799
3,Used,Street,2008,Harley Davidson,XL1200,Wisconsin,22433,4499
4,Used,Street,2008,Harley Davidson,XL1200,Wisconsin,22856,3999


In [369]:
cols = ['year', 'mileage', 'price']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df = df.dropna()

In [370]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6997 entries, 0 to 7483
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   condition  6997 non-null   object 
 1   type       6997 non-null   object 
 2   year       6997 non-null   float64
 3   brand      6997 non-null   object 
 4   model      6997 non-null   object 
 5   state      6997 non-null   object 
 6   mileage    6997 non-null   float64
 7   price      6997 non-null   float64
dtypes: float64(3), object(5)
memory usage: 492.0+ KB


In [371]:
df.describe()

,year,mileage,price
count,6997.000000,6.997000e+03,6997.000000
mean,2017.670430,6.682272e+03,11548.525225
std,4.819854,1.203950e+05,8193.214931
min,1965.000000,0.000000e+00,1.000000
25%,2017.000000,0.000000e+00,5349.000000
50%,2020.000000,0.000000e+00,9595.000000
75%,2020.000000,1.386000e+03,15990.000000
max,2021.000000,1.000000e+07,189989.000000


In [372]:
mask = df['price'] <= 1000
df[mask].shape

(8, 8)

In [373]:
df[mask]

,condition,type,year,brand,model,state,mileage,price
5491,Used,Street,1996.0,Kawasaki,750 SS,Georgia,1.0,800.0
5492,Used,Street,1998.0,Kawasaki,900 STX,Georgia,0.0,880.0
5493,Used,Street,1995.0,Kawasaki,ZXI 750,Georgia,1.0,750.0
5494,Used,Street,1995.0,Kawasaki,750,Georgia,1.0,750.0
5495,Used,Street,1995.0,Kawasaki,ZXI 750,Georgia,1.0,550.0
5496,Used,Street,1990.0,Kawasaki,300,Georgia,0.0,650.0
5497,Used,Street,1994.0,Kawasaki,SUPER SHERPA KL 250,Georgia,0.0,1000.0
6881,Used,Street,2015.0,Harley Davidson,XL1200,South Carolina,2007.0,1.0


In [374]:
mask = df['price'] >= 75000
df[mask].shape

(1, 8)

In [375]:
# Filter out high and low prices
# Filter out older than 1980
# Filter out high mileage?

# Add in plotting for cleaning

high_price = df['price'] >= 75000
low_price  = df['price'] <= 1000
old_bike   = df['year']  <= 1980

mask = df[high_price | low_price | old_bike].index
df = df.drop(mask)

In [376]:
df.isnull().sum()

condition    0
type         0
year         0
brand        0
model        0
state        0
mileage      0
price        0
dtype: int64

In [377]:
df.head()

,condition,type,year,brand,model,state,mileage,price
0,Used,Street,2006.0,Harley Davidson,FXS,Wisconsin,30225.0,6899.0
1,Used,Street,2015.0,Harley Davidson,XG500,Wisconsin,2621.0,3999.0
2,Used,Street,2015.0,Harley Davidson,XG750,Wisconsin,6314.0,4799.0
3,Used,Street,2008.0,Harley Davidson,XL1200,Wisconsin,22433.0,4499.0
4,Used,Street,2008.0,Harley Davidson,XL1200,Wisconsin,22856.0,3999.0


In [ ]:
# Add in is New col and remove condition
bikes['is_new'] = bikes['condition']

In [287]:
pd.get_dummies(df['condition'], drop_first=True)

,Used
0,1
1,1
2,1
3,1
4,1
...,...
7479,1
7480,0
7481,0
7482,0


In [313]:
bikes = pd.get_dummies(df.drop(['state', 'model'], axis=1))

In [314]:
bikes

,year,mileage,price,condition_New,condition_Used,type_Dirt,type_Scooter,type_Street,type_Trike,brand_BMW,...,brand_Genuine Scooter,brand_Harley Davidson,brand_Honda,brand_KTM,brand_Kawasaki,brand_Kymco,brand_Suzuki,brand_Triumph,brand_Vespa,brand_Yamaha
0,2006.0,30225.0,6899.0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2015.0,2621.0,3999.0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2015.0,6314.0,4799.0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2008.0,22433.0,4499.0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2008.0,22856.0,3999.0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7479,2017.0,0.0,5500.0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
7480,2021.0,500.0,8399.0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
7481,2021.0,500.0,5399.0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
7482,2019.0,500.0,7499.0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


# Model

In [319]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [320]:
# Train, Test, Validate
X, y = bikes.drop('price',axis=1), bikes['price']

# hold out 20% of the data for final testing
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Split 20% for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=99)

In [321]:
m = LinearRegression()
m.fit(X_train,y_train)

# Return the coefficient of determination R^2 of the prediction.
m.score(X_train,y_train)

0.49189297960625233

In [322]:
coefs = list(zip(X_train.columns, m.coef_))
coefs

[('year', 472.1120762541751),
 ('mileage', 0.004688000784492597),
 ('condition_New', 1166.557896600607),
 ('condition_Used', -1166.5578966006065),
 ('type_Dirt', -3611.1804819002814),
 ('type_Scooter', -4679.87170182914),
 ('type_Street', 2292.780690516582),
 ('type_Trike', 5998.271493212831),
 ('brand_BMW', 6023.945234089523),
 ('brand_Can Am', 5998.271493212828),
 ('brand_Ducati', 2551.656073588233),
 ('brand_Genuine Scooter', -4451.767131619785),
 ('brand_Harley Davidson', 7290.460684219645),
 ('brand_Honda', -1112.7237889158391),
 ('brand_KTM', -3148.9487701059716),
 ('brand_Kawasaki', -2407.5015091061796),
 ('brand_Kymco', -4437.726094064906),
 ('brand_Suzuki', -2742.0572536297414),
 ('brand_Triumph', 356.41550699053886),
 ('brand_Vespa', -1725.031081825348),
 ('brand_Yamaha', -2194.9933628329945)]

In [329]:
y_pred = m.predict(X_test)

# will predict withing this much
mean_absolute_error(y_test, y_pred)

3933.477553407862

# Add Poly Features

In [332]:
from sklearn.preprocessing import PolynomialFeatures
p = PolynomialFeatures(degree=2)
x_train_poly = p.fit_transform(X_train)
m.fit(x_train_poly,y_train)
m.score(x_train_poly,y_train)

0.0

In [336]:
from sklearn.linear_model import LassoCV

In [337]:
m = LassoCV()
p = PolynomialFeatures(degree=2)
m.fit(p.fit_transform(X_train),y_train)
m.score(p.transform(X_train),y_train)

0.0